In [1]:
# pip install numpy
# pip install pandas
# pip install wheel
# pip install pandas

In [2]:
#Content based filtering

import numpy as np
import pandas as pd
from pandas import DataFrame
import sklearn
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel


In [3]:
#pip install nltk


In [4]:
import time
start_time = int(round(time.time() * 1000))

In [5]:
ratings_dataset = pd.read_csv('/Users/priya/Desktop/tutorials/ratings_Beauty.csv')
products_dataset = pd.read_csv('/Users/priya/Desktop/tutorials/dataset_final.csv')
pr = ratings_dataset.drop_duplicates(['ProductId'])
pdes = products_dataset[['title', 'image', 'asin', 'description']]
products_n  = pd.merge(pr, pdes, left_on= 'ProductId', right_on= 'asin')

/usr/local/Homebrew/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
products_n.head(5)

,UserId,ProductId,Rating,Timestamp,title,image,asin,description
0,ABVND860D55AJ,7535842801,5.0,1375574400,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
1,A1TYR5GO9YE60D,9790787006,5.0,1334275200,Jenna Jameson Heartbreaker Perfume for women 3...,['https://images-na.ssl-images-amazon.com/imag...,9790787006,['Jenna Jameson Heartbreaker Perfume for women...
2,A8OWTSUEDOYLM,B000052YMO,5.0,1366502400,Carmol Enhanced Moisturizing Cream 3 oz (85 g),[],B000052YMO,['<P><B>CARMOL 20:<BR></B><UL><LI>20% Carbamid...
3,A8OWTSUEDOYLM,B000052YMO,5.0,1366502400,Carmol Enhanced Moisturizing Cream 3 oz (85 g),[],B000052YMO,['<P><B>CARMOL 20:<BR></B><UL><LI>20% Carbamid...
4,A3F1VMAU8SBNYA,B00005308B,4.0,1357084800,Old Spice High Endurance Anti-Perspirant &amp;...,['https://images-na.ssl-images-amazon.com/imag...,B00005308B,['Using this is like chumming the air for pret...


In [7]:
products_wd = products_n[products_n['description'].notnull()].copy()
products_wd = products_wd[products_wd['description'].map(len) >5]    #FILTERING

products_wd.reset_index(drop=True, inplace=True)

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_des = tf.fit_transform(products_wd['description'])

In [8]:
indices_n = pd.Series(products_wd['title'])
inddict = indices_n.to_dict()


In [9]:
inddict = dict((v,k) for k,v in inddict.items())

In [10]:
from sklearn.metrics.pairwise import euclidean_distances
D = euclidean_distances(tfidf_des)

In [11]:
def recommend_euclidean_distance(title):
    ind = inddict[title]
    distance = list(enumerate(D[ind]))
    distance = sorted(distance, key=lambda x: x[1])
    distance = distance[1:51]
    
    products_index = [i[0] for i in distance]

    return products_wd.iloc[products_index]

In [12]:
#df=pd.DataFrame(recommend_euclidean_distance("7535842801"))
#list(set(df.title))

print()

res = pd.DataFrame(recommend_euclidean_distance("Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6"))
print(res["title"].head(10))

print()
print()

computational_time = int(round(time.time() * 1000)) - start_time
print('\n\nTime taken for execution of Phase 4 :' ,computational_time, 'milliseconds')


1186         Une Olive en Provence Soap White XL (5.29oz)
1081             Avon Skin So Soft Winter Soft Hand Cream
800     La Mer Skincare Set 3 pcs(The Moisturizing Cre...
733      Mary Kay Forever Orchid Eau de Toilette ~ 1.7 Oz
648     Kracie Peach Body Wash (580mL) with Refill Pac...
1309    Cotton Clouds Rainbow Color 60 Count (12 Bags)...
1350    Josie Maran Whipped Argan Oil Intensive Hand C...
1288         Savannah Bee Company Your Majesty Travel Kit
895       Stay On Satin Large Scarf - Style 779-B - Black
573            American Crew Fiber (Pack of 4) - 3oz each
Name: title, dtype: object




Time taken for execution of Phase 4 : 4815 milliseconds


In [13]:
# Hybrid - content + collaborative

In [14]:
#pip install scikit-surprise


In [15]:
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
import warnings; warnings.simplefilter('ignore')
from pandas import DataFrame
import sklearn
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from surprise.model_selection import cross_validate
from surprise.model_selection import cross_validate, train_test_split

In [30]:
ratings_dataset = pd.read_csv('/Users/priya/Desktop/tutorials/ratings_Beauty.csv')
products_dataset = pd.read_csv('/Users/priya/Desktop/tutorials/dataset_final.csv')
pr = ratings_dataset[['UserId', 'ProductId', 'Rating']]
pdes = products_dataset[['title', 'image', 'asin', 'description']]
books_n  = pd.merge(pr, pdes, left_on= 'ProductId', right_on= 'asin')

books_wd = books_n[books_n['description'].notnull()].copy()
books_wd = books_wd[books_wd['description'].map(len) >5]
books_wd.reset_index(drop=True, inplace=True)
books_wd.head()
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_des = tf.fit_transform(books_wd['description'])
from sklearn.metrics.pairwise import linear_kernel

# comping cosine similarity matrix using linear_kernal of sklearn
cosine_sim_des = linear_kernel(tfidf_des, tfidf_des)
indices_n = pd.Series(books_wd['title'])
inddict = indices_n.to_dict()
#changing the selection of books from index to isbn
inddict = dict((v,k) for k,v in inddict.items())
#euclidean distance
from sklearn.metrics.pairwise import euclidean_distances
D = euclidean_distances(tfidf_des)
def recommend_euclidean_distance(title):
    ind = inddict[title]
    distance = list(enumerate(D[ind]))
    distance = sorted(distance, key=lambda x: x[1])
    distance = distance[1:50]
    #Get the books index
    books_index = [i[0] for i in distance]

    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_wd.iloc[books_index]
recommend_euclidean_distance('Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6')


,UserId,ProductId,Rating,title,image,asin,description
1,A26TLFFBFPEOKJ,7535842801,4.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
2,A2Q8QWVIZM213M,7535842801,3.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
3,A3C5DYTUXUCL0S,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
4,A3DNSZT2HIB30J,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
5,AA0ACUPEPT1RN,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
6,A16UTJN8DODY6N,7535842801,1.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
7,ACB9GSMY2GY0O,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
8,ANNS6DSY390Z2,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
9,A39SBRLJ1EQX1R,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...
10,A3A2HRW0GR4EWU,7535842801,5.0,Mary Kay Satin Hands Hand Cream Travel MINI Si...,[],7535842801,['Mary Kay Satin Hands Peach Hand Cream Travel...


In [32]:
# Hybrid - content + collaborative

In [34]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

import time
start_time_hyb = int(round(time.time() * 1000))

# for Surprise, we only need three columns from the dataset
products_n = pd.read_csv("/Users/priya/Desktop/tutorials/Dataset_merged2.csv")
data = products_n[['UserId', 'ProductId', 'Rating']]
#data.head()
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data, reader=reader)


# train-test-split
trainset, testset = train_test_split(data, test_size=.2)

# instantiate SVD and fit the trainset
svd = SVD()
svd.fit(trainset)

#products_n[products_n['UserId'] == 1]
svd.predict(1,7535842801)

# taking a look at the first 3 rows of our test set
predictions = svd.test(testset)
predictions[:3]


def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan
    

indices_map = products_n.set_index('id')
products_n['Rating'] = products_n['Rating'].apply(convert_int)

def hybrid(UserId,title):
    ind = inddict[title]
    distance = list(enumerate(D[ind]))
    distance = sorted(distance, key=lambda x: x[1])
    distance = distance[1:50]
    products_index = [i[0] for i in distance]
    
    
    products = products_n.iloc[products_index][['title', 'description', 'Rating', 'UserId', 'id']]
    
    products['est'] = products['id'].apply(lambda x: svd.predict(UserId,indices_map.loc[x]['ProductId']).est)
    products = products.sort_values('est', ascending=False)
    return products.head(50)

df=pd.DataFrame(hybrid(7535842801,'Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6'))
#df=pd.DataFrame(hybrid(9790787006,'Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6'))

df=df.drop_duplicates(['title'])

df[['title']].head(10)


,title
13226,Pro Salon Hairdressing Haircutting Gown Barber...
13853,FOONEE Rhinestones Nail Art Gems Mixed Colours...
8937,Lustrasilk Moisture Max Mega-Moist Oil Moistur...
11355,LIP INK Shine Moisturizer Organic Vegan Lip Gl...
8938,"TRESemme 24 Hour Body Weightless Creme, 5 oz"
14,Mary Kay Satin Hands Hand Cream Travel MINI Si...
14837,Bath and Body Works HONEY AUTUMN APPLE Shower ...
11347,Joewell S2 5.5&quot; Mega Deal Shears / Scissors
9638,Clinique 6 Pieces Makeup Set with a Nice Bag
11351,Joewell Skull Taboo Mega Deal 5.0&quot; Shears...
